<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

In [2]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
query_3_1 = f'''select
                count(id)
                from public.vacancies
            '''

In [51]:
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\3851344990.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
query_3_2 = f'''select
            count(name)
            from public.employers
         '''

In [7]:
df = pd.read_sql_query(query_3_2, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\3588252056.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
query_3_3 = f'''select
            count(name)
            from public.areas
         '''

In [9]:
df = pd.read_sql_query(query_3_3, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\2018224393.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
query_3_4 = f'''select
            count(name)
            from public.industries
         '''

In [11]:
df = pd.read_sql_query(query_3_4, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\2172479966.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_4, connection)


,count
0,294


***

База данных содержит сведения о 49197 вакансиях от 23501 работодателя из 1362 регионов в 294 различных сферах деятельности.

По состоянию на 1 октября 2021 года в России насчитывается 1118 городов, следовательно можно предположить что в базе данных имеются вакансии за пределами России.

В среднем на каждого работодателя приходится 2,09 вакансии. Скорее всего в базе представлены как крупные фирмы, предлагающие несколько вакансий, так и мелкие компании с одним предложением.

Количество различных сфер деятельности значительно меньше количества работодателей. В среднем на каждую сферу деятельности приходится около 80 работодателей. Вероятнее всего распределение работодателей по сферам деятельности неравномерное - существуют как популярные сферы деятельности так и достаточно редкие.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
query_4_1 = f'''select
                  a.name area,
                  count(v.name) cnt
               from
                  public.vacancies v
               join public.areas a on v.area_id = a.id
               group by area
               order by cnt desc
               limit 5
            '''

In [13]:
df = pd.read_sql_query(query_4_1, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\347208320.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_1, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
query_4_2 = f'''select
                    count(*)
                from
                    public.vacancies
                where salary_from is not null or salary_to is not null
            '''

In [15]:
df = pd.read_sql_query(query_4_2, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\2168503709.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_2, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
query_4_3 = f'''select
                    round(avg(salary_from), 0) as  from,
                    round(avg(salary_to), 0) as to
                from
                    public.vacancies
            '''

In [17]:
df = pd.read_sql_query(query_4_3, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\2222372203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_3, connection)


,from,to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
query_4_4 = f'''select distinct
                  schedule,
                  employment,
                  count(name)
               from
                  public.vacancies
               group by schedule, employment
               order by 3 desc
            '''

In [19]:
df = pd.read_sql_query(query_4_4, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\2981602351.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_4, connection)


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
query_4_5 = f'''select
                  experience,
               count(id)
               from
                  public.vacancies
               group by 1
               order by 2
            '''

In [21]:
df = pd.read_sql_query(query_4_5, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\2601166264.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_4_5, connection)


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Около трети вакансий (28,85%) предлагается в пяти регонах: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы. Причём на Москву приходится 10,84% всех представленных вакансий. Список регионов подтверждает предыдущий вывод о том, что база охватывает не только регионы России. Отношение количества предложенных в регионах вакансий к населению этих регионов показывает иное распределение регионов: Новосибирск, Минск, Алматы, Санкт-Петербург, Москва. Можно сделать вывод, что база содержит достаточно подробную выборку вакансий по различным регионам, а не центрированна на каком-то одном.

Менее половины (48,93%) работодателей указывают предлагаемую зарплату. Вероятно это связано с желанием работодателей оценить уровень кандидата при собеседовании и исходя из этого предлагать зарплату. Можно предположить, что значительная часть вакансий представленных в базе относится к высокотехнологичным или требующим специальных навыков видам деятельности.

Среднее значение зарплаты составляет: для нижней границы - 71065.0 рублей, для верхней границы - 110537.0 рублей. Учитывая, что более половины работодателей не указывают зарплату, а также достаточно большое количество регионов и возможных видов деятельности данные сведенья мало информативны. Уровень зарплаты стоит рассматривать для вакансий имеющих схожий набор ключевых навыков.

Второе по популярности сочетание типа рабочего графика и типа трудоустройства: ['Удаленная работа' 'Полная занятость']. Такое сочетание встречается у 15,86% вакансий. Можно предположить, что в базе данных представлено достаточно много вакансий имеющих отношение к IT, хотя это могут быть и вакансии связанные с обзвонами или ответами на звонки.

Наиболее популярное требование к опыту работы среди представленных вакансий от 1 года до 3 лет. Более половины работодателей (53,16%) ищут соискателей имеющих начальный опыт работы.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
query_5_1 = f'''(select
                  e.name employers,
                  count(v.id)
               from
                  public.vacancies v
               join public.employers e on v.employer_id=e.id
               group by 1
               order by 2 desc
               limit 1)
               union all
               (select
                  e.name employers,
                  count(v.id)
               from
                  public.vacancies v
               join public.employers e on v.employer_id=e.id
               group by 1
               order by 2 desc
               offset 4
               limit 1)
            '''

In [23]:
df = pd.read_sql_query(query_5_1, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\1792814825.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_1, connection)


,employers,count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
query_5_2 = f'''select
                    a.name,
                    count(distinct e.id) cnt_employers,
                    count(distinct v.id) cnt_vacancies
                from
                    public.areas a
                left join public.employers e on a.id=e.area
                left join public.vacancies v on a.id=v.area_id
                where v.id is null
                group by a.name
                order by 2 desc
                limit 1
             '''

In [25]:
df = pd.read_sql_query(query_5_2, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\3341481329.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_2, connection)


,name,cnt_employers,cnt_vacancies
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
query_5_3 = f'''select
                    e.name,
                    count(distinct v.area_id)
                from
                    public.employers e
                join public.vacancies v on e.id=v.employer_id
                group by 1
                order by 2 desc
                limit 1
             '''

In [27]:
df = pd.read_sql_query(query_5_3, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\2398953733.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_3, connection)


,name,count
0,Яндекс,181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
query_5_4 = f'''select
                    count(e.id)
                from
                    public.employers e
                left join public.employers_industries ei on e.id=ei.employer_id
                where industry_id is null
             '''

In [29]:
df = pd.read_sql_query(query_5_4, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\3114411492.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_4, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
query_5_5 = f'''select
                    e.name
                from
                    public.employers e
                join public.employers_industries ei on e.id=ei.employer_id
                join public.industries i on ei.industry_id=i.id
                group by e.id
                having count(i.name) = 4
                order by 1
                limit 3
             '''

In [31]:
df = pd.read_sql_query(query_5_5, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\1650419958.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_5, connection)


,name
0,101 Интернет
1,21vek.by
2,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
query_5_6 = f'''select
                    count(ei.employer_id)
                from
                    public.industries i
                join public.employers_industries ei on i.id=ei.industry_id
                and i.name = 'Разработка программного обеспечения'
            '''

In [33]:
df = pd.read_sql_query(query_5_6, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\1169039133.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_6, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%' \
    + 'D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
table = page.find_all('table', class_="standard sortable")

table = str(table)
table = table.split('"')

city_ls = []
for i in range(len(table)):
    if table[i] == ' title=' and table[i+1][-1].isalpha():
        city_ls.append(table[i+1])

city_tpl = tuple(city_ls)
print(city_tpl)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [35]:
query_5_7 = f'''select
                    a.name area,
                    count(v.id) count_vacancies
                from
                    public.vacancies v
                join public.areas a on v.area_id = a.id
                    and a.name in {city_tpl}
                join public.employers e on v.employer_id = e.id
                    and e.name = 'Яндекс'
                group by a.name
                union all
                select
                    'total',
                    count(v.id)
                from
                    public.vacancies v
                join public.areas a on v.area_id = a.id
                    and a.name in {city_tpl}
                join public.employers e on v.employer_id = e.id
                    and e.name = 'Яндекс'
                order by count_vacancies
            '''

In [36]:
df = pd.read_sql_query(query_5_7, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\3129374555.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_5_7, connection)


,area,count_vacancies
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

В пятерке лидеров по количеству вакансий находятся IT-компания («Яндекс»), провайдер интернет услуг («Ростелеком»), два представителя банковского сектора («Тинькофф», «СБЕР») и добывающая компания («Газпром нефть»). Всем им в той или иной степени могут требоваться IT-специалисты.

В базе присутствуют регионы, в которых нет вакансий, но в которых зарегестрированы работодатели. Среди таких регионов наибольшее количество работодателей зарегестрировано в регионе Россия. Вероятно часть работодателей имеют филиалы в различных городах, поэтому в графе регион регистрации указывают страну.

Лидером среди работодателей по географии предлагаемых вакансий является «Яндекс». Его вакансии предлагаются в 181 регионе. В базе представлено 192 работодателя которые предлагают вакансии в 10 и более регионах.

Более трети работодателей (8419) не указывают сферу деятельности. Вероятно работодатели считают, что название компании говорит само за себя либо что сфера деятельности понятна из наименования вакансии и набора требуемых навыков.

В базе представлено 1136 компаний указавших четыре сферы деятельности. Чаще всего указываются сферы деятельности относящиеся к различным стадиям жизненого цикла продукта (разработка, изготовление, продвижение/продажа, ремонт). Это говорит о том, что в базе присутствуют крупные компании способные обеспечить полный жизненый цикл своей продукции.

Более 15% (3553) работодателей явно указали в качестве сферы деятельности разработку программного обеспечения. Учитывая, что у трети работодателей совсем не указана сфера деятельности можно предположить что достаточно много предлагаемых в базе вакансий может быть связана с IT-сферой. Но лучше такую оценку проводить по требуемым ключевым навыкам и наименованию вакансии так как в других сферах деятельности тоже может быть потребность в IT-специалистах.

Компания «Яндекс» предлагает от 21 до 54 вакансий в городах-миллионниках России, всего 485 вакансий. В данной компании порядка 18000 сотрудников, количество вакансий в городах-миллионниках составляет 2,69% от этого числа. Можно предположить, что это количество вакансий связано с восполнением ушедших сотрудников. Чтобы сделать адекватный вывод о текучести кадров стоит подсчитать общее количество вакансий компании «Яндекс». Для этого воспользуемся второй частью нашего запроса, подсчитывающей общее число вакансий, и уберем условие нахождения вакансии в городах-миллионниках. В результате получим, что компания «Яндекс» предлагает 1993 вакансии, а это состовляет 10,74% от общего числа сотрудников. Для IT-сферы нормальным значением текучести кадров считается уровень в 8-10%. Интересно было бы рассмотреть показатель текучести персонала в динамическом разрезе. То есть, сравнить его с соответствующими показателями за более ранние периоды. Это позволит оценить насколько правильно выстроена кадровая политика в компании.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [54]:
query_6_1 = f'''select
                    count(id)
                from
                    public.vacancies
                where name ilike '%data%' or name ilike '%данн%'
            '''

In [55]:
df = pd.read_sql_query(query_6_1, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\485020324.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
query_6_2 = f'''select
                    count(id)
                from public.vacancies
                where ((
                    name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%'
                    )
                    or (
                    name like '%ML%' and name not like '%HTML%'
                    ))
                    and (name ilike '%junior%' 
                    or experience ilike '%нет опыта%' 
                    or employment ilike '%стажировка%')
            '''

In [40]:
df = pd.read_sql_query(query_6_2, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\3641650667.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
query_6_3 = f'''select
                    count(id)
                from public.vacancies
                where ((
                    name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%'
                    )
                    or (
                    name like '%ML%' and name not like '%HTML%'
                    ))
                    and(key_skills ilike '%SQL%'
                    or key_skills ilike '%postgres%')
            '''

In [42]:
df = pd.read_sql_query(query_6_3, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\1159587760.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
query_6_4 = f'''select
                    count(id)
                from public.vacancies
                where ((
                    name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%'
                    )
                    or (
                    name like '%ML%' and name not like '%HTML%'
                    ))
                    and(key_skills ilike '%Python%')
            '''

In [44]:
df = pd.read_sql_query(query_6_4, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\972386973.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
query_6_5 = f'''select
                    round(avg(length(key_skills) - length(replace(key_skills, chr(9), '')) + 1.0), 2)
                from public.vacancies
                where ((
                    name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%'
                    )
                    or (
                    name like '%ML%' and name not like '%HTML%'
                    ))
                    and key_skills is not null
            '''

In [46]:
df = pd.read_sql_query(query_6_5, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\390248827.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_5, connection)


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
query_6_6 = f'''select
                    experience,
                    round(avg(coalesce((salary_from + salary_to)/2.0, salary_from, salary_to, 0)), 0) avg_salary
                from public.vacancies
                where ((
                    name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%%'
                    )
                    or (
                    name like '%ML%' and name not like '%HTML%'
                    ))
                    and coalesce((salary_from + salary_to)/2.0, salary_from, salary_to, 0) !=0
                    group by experience
            '''

In [48]:
df = pd.read_sql_query(query_6_6, connection)
df

C:\Users\lexpo\AppData\Local\Temp\ipykernel_13968\3561045109.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_6_6, connection)


,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

Примерно 3,6 процента (1771) вакансий в базе имеют отношение к данным. Небольшой процент вакансий может говорить о малом размере базы. Для проверки рапространенности данного вида деятельности стоит провести оценку среди вакансий имеющих отношение к IT-сфере.

В базе данных есть 51 подходящая вакансия для начинающего дата-сайентиста. Это менее 3% от общего числа вакансий, имеющих отношение к данным. Достаточно мало вакансий возникают вопросы по путям получения опыта. Возможно участие/победы соискателя в хакатонах стоит расценивать как начальный опыт.

Для 201 вакансии на позицию DS в качестве ключевого навыка указан SQL или postgres. Относительно количества вакансий, отвечающих выбранному критерию отнесения вакансии к DS, навыки работы с базами данных требуются в 41,9% случаев. Стоит учесть, что не для всех вакансий указываются ключевые навыки, некоторые работодатели подразумевают наличие навыков в наименовании самой вакансии.

Для 351 вакансии на позицию DS в качестве ключевого навыка указано знание Python. Относительно количества вакансий, отвечающих выбранному критерию отнесения вакансии к DS, знание Python требуется в 73,1% случаев. Действительно Python является наиболее распространненым языком программирования в DS, однако, встречаются вакансии требующие знание С++ или Java. Также стоит учитывать, что иногда требование к ключевым навыкам подразумевается в наименовании вакансии.

Для DS вакансий в среднем указывается 6.41 ключевых навыков. Работодатели наравне с общими навыками ожидают и знания из конкретных областей. Например, кроме знания Python указывается владение библиотками Pandas, Numpy и другие. Возможно для рекомендательной системы стоит заострить внимание на основных навыках, а знание в конкретных областях не учитывать при подборе вакансий. То есть, например, соискатель может явно не указать владение библиотеками Python считая это само собой разумеющимся.

Дата-сайнтист с опытом работы от 3 до 6 лет может рассчитывать на среднюю зарплату: 243115 рублей. Не смотря на то, что не все работодатели указывают уровень зарплат данный показатель может использоваться для оценки уровня зарплаты в привязке к опыту работы. Кроме опыта работы можно рассмотреть зависимость от региона в котором предложена вакансия. Интересно будет посмотреть зависимость от типа рабочего графика.

# Общий вывод по проекту

В нашем распоряжении на первый взгляд достаточно большая подборка данных на 49197 вакансий. Однако согласно википедии среднее дневное количество вакансий на HeadHunter (hh.ru) превышает 933 тыс. Общее количество вакансий на позицию Data Scientist не вилико - всего 480 вакансий, что также приводит к мысли о малом размере подборки данных. Кроме того в данных присутствуют и вакансии не имеющие отношения к IT сфере. Встречаются вакансии регионом которых указана страна и не все из них предлагают удаленную работу. Также мы увидели ннизкую среднюю зарплату в Екатеринбурге для вакансий Data Scientist без опыта работы, при достаточно высокой зарплате для других категорий опыта. Все это говорит о наличии выбросов, для продолжения работы можно предложить провести очистку данных. Так же можно рекомендовать рассмотреть категоризацию ключевых параметров. Например, сгруппировать знание специализированных библиотек со знанием языка. Ведь соискатель может указать только знание Python подразумевая знание библиотек. Думаю, что для рекомендательной системы такое допущение приемлемо;

по результатам анализа предложенных в подборке вакансий Data Scientist можно сказать, что для начала карьеры стоит рассмотреть переезд в Санкт-Петербург, Новосибирск или Москву. Возможно стоит рассмотреть Нижний Новгород или Казань, но данные по зарплате в этих городах отсутствуют. В остальных городах очень мало вакансий для соискателей без опыта или низкий уровень зарплаты. Стоит отметить, что с ростом опыта растет и количество предлагаемых вакансий с типом рабочего графика "Удаленная работа" и город проживания уже не играет решающей роли.